# My notes of Recurent Neural Network Course
coursera https://www.coursera.org/learn/nlp-sequence-models/

In [1]:
import numpy as np

# RNN (week 1)
## Notation
Representing words
- vocabulary / dictionary 
  - common size `~30-40k`
  - one-hot representation
  - token for `<unknown>` words which is not in vocabulary

## Why don't we use standard NN here?
- input and output vectors could have different lengths in different examples
- doesn't share features learned in different possitions
- very large input layer ($10k * max(T_x$))

## RNN

input:

- $a^{<t-1>}$ and $x^{<t>}$
- $a^{<0>}$ usually zero or random values
- $a^{<t>}$ come from NN on t-step to the t+1 step

output:

- $y^{<t>}$

calculation:

$$
a^{<t>} = g_1(W_{aa} a^{<t-1>} + W_{ax} x^{<t>} + b_a)
$$
$g_1$ - usually tanh/relu

$$
\hat{y}^{<t>} = g_2(W_{ya} a^{<t>} + b_y)
$$
$g_2$ - sigmoid

we could collapse for simpler notation

$$
W_{aa} a^{<t-1>} + W_{ax} x^{<t>} == W_a [ a^{<t-1>} ; x^{<t>} ]
$$

problems:

- we don't have information about upcoming events (words) so sentense which very similar at first part could be completely different on the last part what could influent on result of classifier (possible solution BRNN - bidirectional RNN)

## Loss function
$$
L^{<t>}(\hat{y}^{<t>}, y^{<t>}) = - y^{<t>} \log{\hat{y}^{<t>}} - (1 - y^{<t>}) \log{(1 - \hat{y}^{<t>})}
$$
$$
L(\hat{y}^{<t>}, y^{<t>}) = \sum_{t=1}^{T_y}{L^{<t>}(\hat{y}^{<t>}, y^{<t>})}
$$

## Forward and Backpropagation through time

main difference between common is $a^{<t>}$ which pass back and forth "through time"

## Type of RNN. Many-to-Many ($T_x = T_y$)
Example: Entity detection in text

just ordinar RNN, where 

## Type of RNN. Many-to-One
output just on last node. Example: Classify movies review,

## Type of RNN. One-to-Many
input only 1st node and the result just generate
Example: Music generation

## Type of RNN. Many-to-Many ($T_x \neq T_y$)
Example: Translation

1st part look like Many-to-One (encoder) and 2nd part as One-to-Many (decoder)

## Language model and sequence generation
- use `RNN` define chance of each word in sentence
- get large corpus of text
  - `<EOS>` - end of sentence
  - `<UNK>` - uknown word
- tokenize

### the model

first input vector is zero $ x^{<0>} = 0 $. Upcoming input vectos are words from real sentence. so we predict change of new word $\hat{y}^{<t>}$ based on previous one $x^{<t>} = y^{<t - 1>}$ which is sample based on $\hat{y}^{<t-1>}$ probability. Where $y$ are probabilities of each word from corpus. The last $y^{<t>}$ should be `<EOS>`.

Softmax loss function
$$
L(\hat{y}^{<t>}, y^{<t>}) = - \sum_i{y_i^{<t>}\log{\hat{y}^{<t>}}}
$$
$$
L = \sum_t{L(\hat{y}^{<t>}, y^{<t>})}
$$
Probability of 3-word sentence
$$
P(y^{<1>}, y^{<2>}, y^{<3>}) = P(y^{<1>}) P(y^{<2>}|y^{<1>}) P(y^{<3>}|y^{<1>}, y^{<2>})
$$
Character-level language model

## Problems/Weakness. Vanishing gradients with RNNs
- classical RNN doesn't very good to catch very long dependecies. in other words classic RNN has local influences
- gradient should propagete from all steps of RNN and as result it vanished.
- as alternative we could have exploding of gradient - should apply gradient clipping

## Gated Recurrent Unit (GRU)
based on
- [2014 On the Properties of Neural Machine Translation: Encoder-Decoder Approaches
 by Kyunghyun Cho, Bart van Merrienboer, Dzmitry Bahdanau, Yoshua Bengio](https://arxiv.org/abs/1409.1259)
- [2014 Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling by Junyoung Chung, Caglar Gulcehre, KyungHyun Cho, Yoshua Bengio](https://arxiv.org/abs/1412.3555)

GRU tries solve problem of vanishing gradient and forgetnes useful information for example _singular_ or _plurar_ of one subject.

$c$ - memory unit

$c^{<t>}$ - state of unit in moment $t$

### Simpler version

consider overwriting (candidat for replacing $c^{<t>}$)
$$
\tilde{c}^{<t>} = tanh(W_c [c^{<t-1>}, x^{<t>}] + b_c)
$$ 
gate (value between [0, 1] and most of the time 0 or 1), u - means 'update'
$$
\Gamma_u = \sigma(W_u [c^{<t-1>}, x^{<t>}] + b_u)
$$
it decides whether we update $c$ or hold old one
$$
c^{<t>} = \Gamma_u \tilde{c}^{<t>} + (1 - \Gamma_u) c^{<t-1>}
$$
output
$$
y^{<t>} = softmax(c^{<t>})
$$

and btw in this simpler version $c^{<t>} = a^{<t>}$, memory unit the same as activation value from previous node.

### Full version of GRU
main difference in those:
$$
\tilde{c}^{<t>} = tanh(W_c [\Gamma_r c^{<t-1>}, x^{<t>}] + b_c)
$$
$$
\Gamma_r = \sigma(W_r [c^{<t-1>}, x^{<t>}] + b_r)
$$
$\Gamma_r$ means whether we should take into account previous memory unit.

## Long Short Term Memory (LSTM)
articles:
- [1997 Long Short-term Memory by Sepp Hochreiter and Jürgen Schmidhuber](https://www.researchgate.net/publication/13853244_Long_Short-term_Memory)

$$
\tilde{c}^{<t>} = \tanh(W_c [\Gamma_r a^{<t-1>}, x^{<t>}] + b_c)
$$
update gate
$$
\Gamma_u^{<t>} = \sigma(W_u [a^{<t-1>}, x^{<t>}] + b_u)
$$
forgetness gate
$$
\Gamma_f^{<t>} = \sigma(W_f [a^{<t-1>}, x^{<t>}] + b_f)
$$
new memory cell
$$
c^{<t>} = \Gamma_f^{<t>} * c^{<t-1>} + \Gamma_u^{<t>} * \tilde{c}^{<t>}
$$
output gate
$$
\Gamma_o^{<t>} = \sigma(W_o [a^{<t-1>}, x^{<t>}] + b_o)
$$
$$
a^{<t>} = \Gamma_o^{<t>} * \tanh(c^{<t>})
$$
output
$$
y^{<t>} = softmax (a^{<t>})
$$

### peephole connection
sometimes pass $c^{<t-1>}$ with $a^{<t-1>}$ in formulas of $\Gamma$, [more](https://en.wikipedia.org/wiki/Long_short-term_memory#Peephole_LSTM)

## LSTM vs GRU
- GRU 
  - is much simpler faster
  - we could build much bigger architect and scale
- LSTM
  - much powerful 
  - usually default model to try

## Bidirectional RNN (BRNN)
Problem: should know future to correct interpret current state.

Solution: We create acyclic graph with backward recurent layer:
$$
\overleftarrow{a}^{<t>} \leftarrow \overleftarrow{a}^{<t+1>}, x^{<t>}
$$

$$
\hat{y}^{<t>} = g (W_y [ \overrightarrow{a}^{<t>}, \overleftarrow{a}^{<t>} ] 
$$

where $\overrightarrow{a}^{<t>}$ information from pass,
and $\overleftarrow{a}^{<t>}$ information from future.

## Deep RNNs
- we could stack deep nn on top of $y$ output
- it's hard to see a lot of layer of RNN

# NLP (week 2)
## Word representation
- vocabrary
  - pros
    - sparse one-hot vector 
  - cous
    - it doesn't reflect relations between words so it much harder to generalize
    - big matrixes
- featured representation
  - pros
    - similar things would likely to have similar vector of features
    - reduce side of vector (10k -> 0.3k)
  - cons
    - smarse vector -> dense vector
    - but actually there is no guarantee that we got clear feature because according to linear algebra:
    $$
    \theta_i^T e_j = (A \theta_i)^T (A ^-1 e_j)
    $$
    we could have any valid $A$ linear degenerate transformation
    
## Using word embeddings
- could easily apply transfer learning 
  - learn word embedding on large text courpus (1-100b words) 
  - transfer embedding to new task with smaller training set (100k)
    - continue to finetune the word embedding with new data (if you have really big data for traing data set)
- cons
  - less useful for machine translation (why?)
- areas
  - entity recognition
  - text summarization
  - co-reference resolution
  - parsing
  - etc
- less useful (where you have big traing data set in our area - 2nd)
  - language modeling
  - machine translation
- related to:
  - face encoding (represent face in small vector)
    - difference: 
      - goal of face - encode any face
      - goal of nlp - represent relativy fix dictionary but highlight relation and differences between entities

## Properties of word embeddings
$$
e_{man} - e_{woman} \approx e_{king} - e_w
$$

$$
argmax_w( sim(e_w, e_{king} - e_{man} + e_{woman}))
$$

what usually gives 30-70% accuracy, according to sci papers

- `sim` - usually is consine similarty
$$
sim = \frac{u^Tv}{||u|| ||v||}
$$
if u and v very similar inner product ($u^Tv$) will be very large.

It's named this way because it is `cos` between `u` and `v`

## Learning word embedding
- n-gramms good for language modeling
- but for word embedding ever 1 last word, nearby 1 word or skip-gram are enough

## Word2Vec
[Efficient Estimation of Word Representations in Vector Space - Tomas Mikolov, Kai Chen, Greg Corrado, Jeffrey Dean](https://arxiv.org/abs/1301.3781)
- it based on skip-gram model
- get one `context` word for the `target`
- model
$$O_c E = e_c$$
$E$ - embedding matrix
$e_c$ - embedding vector

$e_c$ -> softmax -> $\hat{y}$

$\hat{y} = p(t|c)$ - probability of target word t in context of c

$$
p(t|c) = \frac{e^{\theta_t^T e_c}}{\sum_{j=1}^{10k}{e^{\theta_j^T e_c}}}
$$

$\theta$ - paramtere associated with output t.
- loss function
$$
L(\hat{y},y) = - \sum_{i=1}^{10k}{y_i \log{\hat{y_i}}}
$$

$y_i$ - one-hot vector which have 0 everywhere and only single 1 for the word
- $p(c)$ we could take into account that some words are more popular then another
### primary problem
computation speed - too many parameters to evaluate probability $p(t|c)$ (softmax)
### workaround
hierarchical softmax. 
- reduce number of calculation from N to log N
- it is not ballanced tree but more popular words lay on lover levels

### Negative sampling
[Distributed Representations of Words and Phrases and their Compositionality - Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado, Jeffrey Dean](https://arxiv.org/abs/1310.4546)
- get single context and single target word - it will be possitive sample
- get k random words from dictionary - they will be negative samples
- k from [5,20] for smaller dataset
- k from [2,5] for large dataset
- model
$$
P(y=1 | c,t) = \sigma(\theta_t^T e_c)
$$
- instead of classifing 10k (size of dictionary) classes, we will have 10k binarry classifers
- empirical distribution for sampling
$$
P(w_i) = \frac{f(w_i)^{3/4}}{\sum_{j=1}^{10k}{f(w_j)^{3/4}}}
$$
where $f(w_j)$ is obsorve frequency of word

## GloVe word vectos
[GloVe: Global Vectors for Word Representation Jeffrey Pennington,   Richard Socher,   Christopher D. Manning](https://nlp.stanford.edu/projects/glove/)
- model
$X_{ij}$ - how much $j$ in context of $i$, $X_{ij} = X_{ji}$
minimize
$$
minimize = \sum_{i=1}^{10k}\sum_{j=1}^{10k}f(X_{ij})(\theta_i^T e_j + b_i + b_j - \log{X_{ij}})^2
$$
$f(X_{ij})$ - heuristic, weighting term which $= 0$ when $X_{ij} = 0$ to "compensate" $\log 0$ (here convension $0 \log 0 = 0$,
also it give more weight for less frequent words

$$
e_{final} = \frac{e_{wt}\theta_w}{2}
$$

## Application. Sentiment classification
word embedding could help in case of small dataset.
- ? (For me it looks like bag of words)
  - model
    - get all words from sentence
    - calculate embedding vector for them
    - avg or sum them 
    - apply softmax to get probability of each classes
  - problem
    - ignore words order (for example "not good" and "good" here would be almost the same)
- many-to-one RNN architecture

## Application. Debiasing word embeddings
[Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings Tolga Bolukbasi, Kai-Wei Chang, James Zou, Venkatesh Saligrama, Adam Kalai](https://arxiv.org/abs/1607.06520)
- example: 
  - _Man -> Computer_Programmer as Woman -> Homemaker_
  - _Father -> Doctor as Mother as Nurse_
- word embeddings can reflect gender, ethnicity, age, sexual orientation and etc
- solution
  - identify bies direction:
  $$
  e_{hi} - e_{she}
  $$
  $$
  e_{male} - e_{female}
  $$
  and etc and avg them.
  - neutralize - for every word that is not definitional, project to get rid of bies (for example: `doctor`, `babysitter`, etc)
  - equalize pairs (`grandmother <-> grandfather`, `girl <-> boy`, etc) there shouldn't be difference between pair words except gender